In [1]:
# System imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
import panel as pn
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact
from panel import widgets
from string import digits
import csv
import json
import numpy as np
import dateparser

pn.extension()

%matplotlib inline

In [2]:
# Local imports
import sys
sys.path.append("../lib2")

from Constants import Constants
from PreprocessingTools import CustomerNameCleaningFunctions
from PreprocessingTools import DateCleaningFunctions
from PreprocessingTools import SpecializedDateCleaningFunctions
from PreprocessingTools import DurationParsing


In [3]:
# Construct tools
debug_level = 0
constants = Constants()
tool_customer = CustomerNameCleaningFunctions(debug_level)
tool_date = DateCleaningFunctions(debug_level)
tool_special_date = SpecializedDateCleaningFunctions(debug_level)
tool_duration_parsing = DurationParsing(debug_level)

In [8]:
constants.DATA_DIR_ANONYMIZED


WindowsPath('../../Resources/02_Anonymized')

In [4]:
# Read anonymized data
atlas                   = pd.read_csv(constants.ANON_ATLAS_FILE_PATH, index_col="Customers")
forecast                = pd.read_csv(constants.ANON_FORECAST_DATA_FILE_PATH, index_col="Organization Name")
revenue2020             = pd.read_csv(constants.ANON_REVENUE2020_FILE_PATH, index_col="Name")
revenue2020A            = pd.read_csv(constants.ANON_REVENUE2020A_FILE_PATH, index_col="Payee Name")
atlas2                  = pd.read_csv(constants.ANON_ATLAS_2_FILE_PATH, index_col="Customers")


FileNotFoundError: [Errno 2] No such file or directory: '..\\..\\Resources\\02_Anonymized\\ATLAS.csv'

In [ ]:
# Clean up ATLAS data

## Invoice Date
atlas["Invoice Date"]           = tool_date.cleanup_date_string_list(atlas["Invoice Date"])

## Invoice Amount
atlas["Invoice Amount"]         = tool_date.cleanup_dollar_string_list(atlas["Invoice Amount"])

## Dates of service
dates_of_service                = tool_special_date.extract_subscription_dates_list(atlas["Dates of service "])
atlas["Service Start"]          = dates_of_service[0]
atlas["Service End"]            = dates_of_service[1]
atlas.drop(columns=["Dates of service "], inplace=True)

## Transform Subscription into a duration to enable math
atlas["Subscription Duration"] = tool_duration_parsing.parse_duration_str_list(atlas["Subscription"])

atlas.head()

In [ ]:
# Clean up 2021 forecast data

## Remove blank columns
forecast.drop(
    columns=["Unnamed: 8", "Unnamed: 9", "Unnamed: 10", "Unnamed: 11", "Unnamed: 12", "Unnamed: 13", "Unnamed: 14", "Unnamed: 15", "Unnamed: 16", "Unnamed: 17", "Unnamed: 18", "Unnamed: 19", "Unnamed: 20", "Unnamed: 21", "Unnamed: 22", "Unnamed: 23", "Unnamed: 24", "Unnamed: 25", "Unnamed: 26", "Unnamed: 27", "Unnamed: 28", "Unnamed: 29", "Unnamed: 30", "Unnamed: 31", "Unnamed: 32", "Unnamed: 33", "Unnamed: 34", "Unnamed: 35", "Unnamed: 36", "Unnamed: 37", "Unnamed: 38", "Unnamed: 39", "Unnamed: 40", "Unnamed: 41", "Unnamed: 42", "Unnamed: 43", "Unnamed: 44", "Unnamed: 45", "Unnamed: 46", "Unnamed: 47", "Unnamed: 48", "Unnamed: 49", "Unnamed: 50", "Unnamed: 51", "Unnamed: 52", "Unnamed: 53", "Unnamed: 54", "Unnamed: 55", "Unnamed: 56", "Unnamed: 57", "Unnamed: 58", "Unnamed: 59", "Unnamed: 60", "Unnamed: 61", "Unnamed: 62", "Unnamed: 63", "Unnamed: 64", "Unnamed: 65", "Unnamed: 66", "Unnamed: 67", "Unnamed: 68", "Unnamed: 69", "Unnamed: 70", "Unnamed: 71", "Unnamed: 72", "Unnamed: 73", "Unnamed: 74", "Unnamed: 75", "Unnamed: 76", "Unnamed: 77", "Unnamed: 78", "Unnamed: 79", "Unnamed: 80", "Unnamed: 81", "Unnamed: 82", "Unnamed: 83", "Unnamed: 84", "Unnamed: 85", "Unnamed: 86", "Unnamed: 87", "Unnamed: 88", "Unnamed: 89", "Unnamed: 90", "Unnamed: 91", "Unnamed: 92", "Unnamed: 93", "Unnamed: 94", "Unnamed: 95", "Unnamed: 96", "Unnamed: 97", "Unnamed: 98", "Unnamed: 99", "Unnamed: 100", "Unnamed: 101", "Unnamed: 102", "Unnamed: 103", "Unnamed: 104", "Unnamed: 105", "Unnamed: 106", "Unnamed: 107", "Unnamed: 108", "Unnamed: 109", "Unnamed: 110", "Unnamed: 111", "Unnamed: 112", "Unnamed: 113"],
    inplace=True)

## Parse dates
forecast["Contract Start Date"]     = tool_date.cleanup_date_string_list(forecast["Contract Start Date"])
forecast["Contract End Date"]       = tool_date.cleanup_date_string_list(forecast["Contract End Date"])

## Parse dollars
forecast["Subscription Fee"]        = tool_date.cleanup_dollar_string_list(forecast["Subscription Fee"])

## Transform Subscription into a duration to enable math
forecast["Subscription Duration"] = tool_duration_parsing.parse_duration_str_list(forecast["Subscription Type"])

forecast.head()

In [ ]:
forecast.dtypes

In [ ]:
# Clean up revenue2020 data

## Parse dates (dates are a little suspicious since it assumes today's day-of-month and year. Only the month value is accurate.)
revenue2020["Month_Dateformat"]     = tool_date.cleanup_date_string_list(revenue2020["Month"])

revenue2020.head()

In [ ]:
# Clean up revenue2020A data

## Remove blank columns
revenue2020A.drop(
    columns=["Unnamed: 8", "Unnamed: 9", "Unnamed: 10"],
    inplace=True)

## Numeric types
revenue2020A["Invoice #"]           = pd.to_numeric(revenue2020A["Invoice #"], errors='coerce')
revenue2020A["Vendor ID"]           = pd.to_numeric(revenue2020A["Vendor ID"], errors='coerce')

## Dollar figures
revenue2020A["Invoice Amount"]      = tool_date.cleanup_dollar_string_list(revenue2020A["Invoice Amount"])
revenue2020A["Amount Paid"]         = tool_date.cleanup_dollar_string_list(revenue2020A["Amount Paid"])

## Parse dates
revenue2020A["Invoice Date"]        = tool_date.cleanup_date_string_list(revenue2020A["Invoice Date"])
revenue2020A["Mail Date"]           = tool_date.cleanup_date_string_list(revenue2020A["Mail Date"])
revenue2020A[" Pymt Received"]      = tool_date.cleanup_date_string_list(revenue2020A[" Pymt Received"])

## Dates of service
dates_of_service            = tool_special_date.extract_subscription_dates_list(revenue2020A["Dates of service "])
revenue2020A["Service Start"]      = dates_of_service[0]
revenue2020A["Service End"]        = dates_of_service[1]
revenue2020A.drop(columns=["Dates of service "], inplace=True)

## Compute service duration
revenue2020A["Service Duration"] = revenue2020A["Service End"] - revenue2020A["Service Start"]

revenue2020A.head()

In [ ]:
# Clean up ATLAS 2 data

## Invoice Date
atlas2["Invoice Date"]          = tool_date.cleanup_date_string_list(atlas2["Invoice Date"])

## Invoice Amount
atlas2["Invoice Amount"]        = tool_date.cleanup_dollar_string_list(atlas2["Invoice Amount"])

## Dates of service
dates_of_service                = tool_special_date.extract_subscription_dates_list(atlas2["Dates of service "])
atlas2["Service Start"]         = dates_of_service[0]
atlas2["Service End"]           = dates_of_service[1]
atlas2.drop(columns=["Dates of service "], inplace=True)

## Transform Subscription into a duration to enable math
atlas2["Subscription Duration"] = tool_duration_parsing.parse_duration_str_list(atlas2["Subscription"])

atlas2.head()

In [ ]:
# Export all files
atlas.to_pickle(constants.PREPROCESSED_ATLAS_FILE_PATH)
forecast.to_pickle(constants.PREPROCESSED_FORECAST_DATA_FILE_PATH)
revenue2020.to_pickle(constants.PREPROCESSED_REVENUE2020_FILE_PATH)
revenue2020A.to_pickle(constants.PREPROCESSED_REVENUE2020A_FILE_PATH)
atlas2.to_pickle(constants.PREPROCESSED_ATLAS_2_FILE_PATH)